# Fitting a model convolved with pseudo-Voigt to 2D neutron scattering data

The data consist in a 2D array, for which the first axis correspond to scattering angle and the second axis to the energy transfer.

This data corresponds to a measurement of protein cetner-of-mass diffusion and internal dynamics obtained using neutron backscattering. The model takes into account the resolution in energy of the instrument $R(q, \omega)$, which is described by a pseudo-Voigt model.

The model is expressed as:
$$
    S(q, \omega) = R(q, \omega) \otimes \beta \left[ 
                   a \mathcal{L}_{\Gamma} + (1 - a) \mathcal{L}_{\gamma + \Gamma} \right] + bkgd
$$

where $\mathcal{L}_{\Gamma}$ is a Lorentzian accounting for center-of-mass diffusion, $\mathcal{L}_{\gamma + \Gamma}$ accounts for internal dynamics convoluted with center-of-mass diffusion and $bkgd$ is a background term.

## Import packages

In [1]:
import os

%matplotlib widget
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib

import ipywidgets as ipw

import numpy as np

import json

import lmfit

## Set the dataset to be fitted

In [2]:
X = np.arange(1024) * 60/1024 - 30
qVals = np.array([0.43594975, 0.56783502, 0.69714328,
                  0.8232508 , 0.94556232, 1.06350556, 1.17653097, 1.28411303,
                  1.38575216, 1.48097673, 1.56934508, 1.65044748, 1.72390798,
                  1.7893861 , 1.84657839])

# assumes the jupyter notebook was run from the 'examples' directory
with open('2D_neutron_vanadium_data.json', 'r') as f:
    vana = json.load(f)
    vanaErr = np.array(vana[1]).reshape((18, 1024))[2:-1]
    vanaData = np.array(vana[0]).reshape((18, 1024))[2:-1]
    
with open('2D_neutron_data.json', 'r') as f:
    expData = json.load(f)
    expErr = np.array(expData[1]).reshape((18, 1024))[2:-1]
    expData = np.array(expData[0]).reshape((18, 1024))[2:-1]

## Fit the resolution function and normalize the dataset

In [3]:
resModel = lmfit.models.PseudoVoigtModel()

resFit = []
for idx, val in enumerate(qVals):
    resFit.append(resModel.fit(vanaData[idx], x=X))
    expData[idx] /= resFit[idx].best_values['amplitude']
    expErr[idx] /= resFit[idx].best_values['amplitude']
    
    # set the model values to the optimized ones and fix them
    for key, val in resFit[idx].best_values.items():
        resModel.set_param_hint(key, value=val, vary=False)

## Plot the experimental dataset

In [4]:
fig, ax = plt.subplots(subplot_kw={'projection': '3d'})

cmap = matplotlib.cm.get_cmap('winter')
norm = matplotlib.colors.Normalize(vmin=qVals[0], vmax=qVals[-1])

for idx, dat in enumerate(expData[::-1]):
    ax.plot(X, dat, qVals[::-1][idx], zdir='y', c=cmap(norm(qVals[::-1][idx])))
    
ax.set_xlabel('$\\rm \hbar \omega ~ [\mu eV]$')
ax.set_ylabel('$\\rm q ~ [\AA^{-1}]$')
ax.set_zlabel('$\\rm S(q, \omega)$')

fig.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Set the model

In [5]:
def lorentz(x, q, beta, a, width, bkgd):
    return beta * a * width * q**2 / (np.pi * (x**2 + (width * q**2)**2)) + bkgd

def jump_diff(x, q, beta, a, width, tau, bkgd): 
    width = width * q**2 / (1 + width * q**2 * tau)
    return beta * a * width / (np.pi * (x**2 + width**2)) + bkgd


model = lmfit.Model(lorentz, independent_vars=['x', 'q']) 
model = model + lmfit.Model(jump_diff, independent_vars=['x', 'q'], prefix='jd_')

# set the parameters starting values and bounds
model.set_param_hint('beta', value=1., min=0., max=np.inf)
model.set_param_hint('a', value=0.8, min=0., max=1)
model.set_param_hint('width', value=15, min=0.1, max=2)
model.set_param_hint('bkgd', value=0.005, min=0., max=1)
model.set_param_hint('jd_beta', value=1., min=0., max=np.inf, expr='beta')
model.set_param_hint('jd_a', value=0.5, min=0., max=1, expr='1 - a')
model.set_param_hint('jd_width', value=30, min=0.1, max=2)
model.set_param_hint('jd_tau', value=1, min=0., max=np.inf)
model.set_param_hint('jd_bkgd', value=0.005, min=0., max=1, expr='bkgd')

## Set the analytic convolutions and create the convolved model

In [6]:
from lmfit.lineshapes import voigt

def conv_lorentz_pvoigt(left, right, params, **kwargs):
    # set left to lorentzian component if not so
    if left.func.__name__ == 'pvoigt':
        tmp = right
        right = left
        left = tmp

    lp = left.make_funcargs(params, kwargs)
    rp = right.make_funcargs(params, kwargs)
    
    # set the q-dependent Lorentzian width
    q = lp['q']
    gamma = lp['width'] * q**2

    amplitude = lp['a'] * lp['beta'] * rp['amplitude']
    center = rp['center']
    sigma_l = gamma + rp['sigma']
    sigma_v = rp['sigma']
    fraction = rp['fraction']
    bkgd = lp['bkgd']

    x = lp['x']

    out = ((1 - fraction) * (voigt(x, amplitude, center, sigma_v, gamma) + bkgd)
           + fraction * lorentz(x, 1, amplitude, center, sigma_l, bkgd))

    return out

def conv_jumpdiff_pvoigt(left, right, params, **kwargs):
    # set left to lorentzian component if not so
    if left.func.__name__ == 'pvoigt':
        tmp = right
        right = left
        left = tmp

    lp = left.make_funcargs(params, kwargs)
    rp = right.make_funcargs(params, kwargs)
    
    # set the q-dependent Lorentzian width
    q = lp['q']
    gamma = lp['width'] * q**2 / (1 + lp['width'] * q**2 * lp['tau'])

    amplitude = lp['a'] * lp['beta'] * rp['amplitude']
    center = rp['center']
    sigma_l = gamma + rp['sigma']
    sigma_v = rp['sigma'] 
    fraction = rp['fraction']
    bkgd = lp['bkgd']
    tau = lp['tau']

    x = lp['x']

    out = ((1 - fraction) * (voigt(x, amplitude, center, sigma_v, gamma) + bkgd)
           + fraction * jump_diff(x, 1, amplitude, center, sigma_l, tau, bkgd))

    return out

# assign the convolution functions to the components
model.left.convolutions['pvoigt'] = conv_lorentz_pvoigt
model.right.convolutions['pvoigt'] = conv_jumpdiff_pvoigt

model._on_undefined_conv = 'raise'

# create the analytically convolved model
model = model << resModel

## Fit the model

In [7]:
res = []
for idx, val in enumerate(qVals):
    res.append(model.fit(expData[idx], x=X, q=val))

## Plot the result

In [8]:
qVal = ipw.SelectionSlider(
           options=qVals.round(2),
           value=qVals.round(2)[0],
           continuous_update=True,
           description='$\\rm q~ [\mathring{A}^{-1}]$'
)


fig, ax = plt.subplots()
ax.plot(X, expData[qVal.index], label='experimental at q=%s [$\\rm \AA^{-1}$]' % qVal.value)
ax.plot(X, res[qVal.index].best_fit, label='model')
ax.set_yscale('log')
ax.set_xlabel('$\\rm \hbar \omega~ [\mu eV]$')
ax.set_ylabel('$\\rm S(q, \omega)$')
ax.legend(frameon=False, loc='upper right')


def update(index):
    ax.lines[0].set_ydata(expData[qVal.index])
    ax.lines[0].set_label('experimental at q=%s [$\\rm \AA^{-1}$]' % qVal.value)
    ax.lines[1].set_ydata(res[qVal.index].best_fit)
    ax.legend(frameon=False, loc='upper right')
    
ipw.interact(update, index=qVal)
    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(SelectionSlider(description='$\\rm q~ [\\mathring{A}^{-1}]$', options=(0.44, 0.57, 0.7, …

<function __main__.update(index)>